In [3]:
%run DataandTokenizer.ipynb
%run Transformer.ipynb
import time
#This are the parameters that define our transformers
layersnum = 4
modeldim = 128
dff = 512 #<- this is for the RELu layers between layers
num_heads = 4 
dropout_rate = 0.1


class Translator(tf.Module):
    def __init__(self, embed, transformer):
        self.embed = embed
        self.transformer = transformer
        checkpoint_path = "./build"

        ckpt = tf.train.Checkpoint(transformer=transformer,
                               optimizer=optimizer)

        ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

        ckpt.restore(ckpt_manager.latest_checkpoint)
    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)
    
    
        ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

        ckpt.restore(ckpt_manager.latest_checkpoint)
    def __call__(self, sentence, max_length=75):
    # input sentence is portuguese, hence adding the start and end token
        data=[]
        dataes=[]
        datapt=[]
        dataes.append(sentence)
        data.append(dataes)
        data.append(datapt)
        sentence=self.embed.tokenize(data)
        encoder_input = sentence[0]
        data2=[]
        data2.append(encoder_input)
        data2.append([[2]])
        # as the target is english, the first token to the transformer should be the
        # english start token.
        start = sentence[0][0][0][tf.newaxis]#WE USE THE IDS IN spanish BECAUSE THEY ARE THE SAME IN PT
        end = sentence[0][0][-1][tf.newaxis] #there could be some padding!
        # `tf.TensorArray` is required here (instead of a python list) so that the
        # dynamic-loop can be traced by `tf.function`.
        output_array = tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True)
        output_array = output_array.write(0, start)

        for i in tf.range(max_length):
            output = tf.transpose(output_array.stack())
            predictions, _ = self.transformer([encoder_input, output], training=False)

          # select the last token from the seq_len dimension
            predictions = predictions[:, -1:, :]  # (batch_size, 1, vocab_size)

            predicted_id = tf.argmax(predictions, axis=-1)
          # concatentate the predicted_id to the output which is given to the decoder
          # as its input.
            output_array = output_array.write(i+1, predicted_id[0])

            if predicted_id == end:
                break

        output = tf.transpose(output_array.stack())
    # output.shape (1, tokens)
        data=[]
        data.append([[2]])
        data.append(output[tf.newaxis][0])
        text = (self.embed.detokenize(data)[1].numpy())[0]
        text=text.decode('UTF-8')
        text=text.removeprefix('[START]')
        
        text=text.removesuffix('[END]')

        # `tf.function` prevents us from using the attention_weights that were
        # calculated on the last iteration of the loop. So recalculate them outside
        # the loop.
        _, attention_weights = self.transformer([encoder_input, output[:,:-1]], training=False)

        return text, attention_weights

        
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = modeldim
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps
learning_rate = CustomSchedule(modeldim)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)